<a href="https://colab.research.google.com/github/a-forty-two/DataSetsForML/blob/master/11_Colored_Images_and_Exporting_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 # DIMENSION THAT YOU WANT TO HARDCODE FOR NN
 # Minimum dimension and filter size?
 # Hyperparameters
 # best dimensions to work with?
 # Image Size -> smaller than 32X32 for colored, 28X28 for grayscale will lose data upon compression
 # Colored Images -> 96,96 spread across 3 dimensions
 # Grayscale images -> 28,28 spread across 1 dimension

 # WHAT IS THE ORDER IN WHICH IMAGE IS COMING TO THE NETWORK?
 # (height,width,depth) -> depth is RGB, also DIMENSION 
 # in NN we call depth as -> CHANNEL_DIMENSION
 # 2 popular channel dimensions are 1 and -1 
 # 1 means that depth is the 1st parameter -> (depth, height, width) -> (3,96,96) CHANNEL_FIRST
 # -1 means that depth is the last paramet -> (height, width, depth)   -> (96,96,3) CHANNEL_LAST
 # -1 -> TensorFlow, 1 -> Pytorch or theano
 
 # RESIZE IMAGE if they are not equal 


In [0]:
# TABULAR DATA -> EXCEL, CSV, JSON, 
# there was always an output column -> y
# rest of the columns/features were tested for their importance 
# select good features and learn on them
# WE NEED TO BUILD OUR OWN DataFrames FOR IMAGES! 

# DUMMY DATASET -> 1000s of images can be using 
!ls

data


In [0]:
!ls data

cartoon  sample_data  snek


In [0]:
# hyperparams
!pwd # linux command to check the path where you are 

/content


In [0]:
!ls /content/data

cartoon  sample_data  snek


In [0]:
path = '/content/data'
IMAGE_DIMS = (224,224,3)
data = [] # X, supposed to be read from folder 
labels = [] # Y
import random
from imutils import paths
allImages = sorted(list(paths.list_images(path)))
random.seed(42)
random.shuffle(allImages)
allImages

['/content/data/snek/000001.png',
 '/content/data/cartoon/000002.png',
 '/content/data/cartoon/000003.png',
 '/content/data/snek/000002.png',
 '/content/data/cartoon/000001.png']

In [0]:
# loop through allImages and create X and Y (input and output)
import cv2
from keras.preprocessing.image import img_to_array
import os
for imagePath in allImages:
  img = cv2.imread(imagePath)
  # images are off different size, RESIZE THEM ALL TO SAME SIZE
  img = cv2.resize(img, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
  image = img_to_array(img)
  data.append(image)
  label = imagePath.split(os.path.sep)[-2] # SECOND LAST ELEMENT iN THE PATHNAME WAS THE CLASSNAME!!!
  labels.append(label)
  print(label)
  print(imagePath)


snek
/content/data/snek/000001.png
cartoon
/content/data/cartoon/000002.png
cartoon
/content/data/cartoon/000003.png
snek
/content/data/snek/000002.png
cartoon
/content/data/cartoon/000001.png


In [0]:
# NORMALIZE the data -> DIVIDE PIXELS by 255! largest pixel is 255! so all values will be 
# scaled between 0 and 1
import numpy as np
from sklearn.preprocessing import LabelBinarizer
data = np.array(data,dtype='float') / 255.0
labels = np.array(labels)
# OneHot, MultiHot, LabelEncoding, we only talked about LabelBinarizer 
# AnkleBoot, Tshirt, Coat -> LabelEncoding for output names!
# LabelBinarizer converts your data into a 1-0 Matrix 
lb = LabelBinarizer()
labels = lb.fit_transform(labels) # A type of ENCODING FOR OUTPUT 
labels

array([[1],
       [0],
       [0],
       [1],
       [0]])

In [0]:
# SINCE ALL DATA IS READY, train_test_split 
from sklearn.model_selection import train_test_split
trainx, testx, trainy, testy = train_test_split(data, labels, test_size=0.2, random_state=42)
len(testy)


1

In [0]:
from keras.preprocessing.image import ImageDataGenerator

# IMAGE AUGMENTATION IS TAKEN CARE AT THIS STEP
aug_fn = ImageDataGenerator(rotation_range=25,width_shift_range=0.1, height_shift_range=0.1, 
                            shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# this is a TF TRANSFORMATION -> only RUN dynamically -> ONLY A LOGIC

In [0]:
from keras.optimizers import Adam
HP_epochs = 100
from keras.applications import VGG16
model = VGG16()
opt = Adam(lr=0.001, decay= 0.001/HP_epochs) # by the time learning is over, we want our LR ~ 0
#model.summary()

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [0]:
HP_batchSize = 32
history = model.fit_generator(aug_fn.flow(trainx, trainy, batch_size=HP_batchSize),
                              validation_data=(testx, testy), epochs = 10)

Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 8.0120 - acc: 0.0000e+00 - val_loss: 16.1182 - val_acc: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 11s 11s/step - loss: 8.0592 - acc: 0.5000 - val_loss: 1.0228e-04 - val_acc: 1.0000
Epoch 3/10
1/1 [==============================] - 11s 11s/step - loss: 8.0592 - acc: 0.5000 - val_loss: 1.0228e-04 - val_acc: 1.0000
Epoch 4/10
1/1 [==============================] - 11s 11s/step - loss: 8.0592 - acc: 0.5000 - val_loss: 1.0228e-04 - val_acc: 1.0000
Epoch 5/10
1/1 [==============================] - 11s 11s/step - loss: 8.0592 - acc: 0.5000 - val_loss: 1.0228e-04 - val_acc: 1.0000
Epoch 6/10
1/1 [==============================] - 11s 11s/step - loss: 8.0592 - acc: 0.5000 - val_loss: 1.0228e-04 - val_acc: 1.0000
Epoch 7/10
1/1 [==============================] - 11s 11s/step - loss: 8.0592 - acc: 0.5000 - val_loss: 1.0228e-04 - val_acc: 1.0000
Epoch 8/10
1/1 [==============================] - 11s 11s/step -

In [0]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
!pwd

/content


In [0]:
!ls

data  jq1.png


In [0]:
# Preparing test image to be passed to model
testImg = cv2.imread('/content/snek2.png')
output = testImg.copy()# right practice, keep a copy safe

# GO THRU THE SAME PROCESSING THAT YOU DID TO PREP THE IMAGE
testImg = cv2.resize(testImg, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
testImg = testImg.astype('float')/255.0
testImg = img_to_array(testImg)
testImg = np.expand_dims(testImg, axis=0)


In [0]:
p = model.predict(testImg) # index of my classification
idx = np.argmax(p)
label = lb.classes_[idx]

In [0]:
label

'cartoon'

In [0]:
# We will build our own VGG model
# It will be smaller, faster
# TRAINING TIME WILL BE MUCH HIGHER -> >8-10 hours
# HOMEWORK : BUILDING A DATASET
# ANIMAL: 5 people in the team-> each building for 1 class
# 250-300 images for each class
# SUPERHERO : Batman, Superman, Shaktiman, SilverSurfer, SamuraiJack 
# 250-300 images for each of the class above
# ANIMALS: Tiger, Lion, Chimp, Gorrilas, Langoors 
# 250-300 images for each of the class above 
# ONCE YOU HAVE COLLECTED IMAGES- they would need to be cleaned manually
# - REPEATED IMAGES SHOULD BE DELETED
# - your search results could contain noise/wrong images
# PNG,JPEG, JPG, BMP, DAT -> use formats that are easily manipulatable 
# AWS S3=> upload all this sample data -> we can download it in GC or anywhere else


In [0]:
model.save('myRainyModel.h5') # EXPORT a model
# TRANSFER LEARNING : ONLY EXPORT WEIGHTS


In [0]:
# i could load this save model anywhere else using
from tensorflow import keras
model = keras.models.load_model('myRainyModel.h5')
# if only weights were exported, create your own model of exactly SAME
# shape as that of trained model, and load the weights 
# model= keras.Sequntial() # model.add(...)
# model.load_weights('weights.pkl')
# monday -> how to build an API out of exported model 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
